In [2]:
%matplotlib inline

import numpy as np
import keras
import matplotlib.pyplot as plt
import pysam
import argparse
import h5py
from array import array
        
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense
from keras.models import Model


In [53]:
#### Filepaths
grch38 = "/seq/references/Homo_sapiens_assembly38/v0/Homo_sapiens_assembly38.fasta"
cram = "/dsde/data/datasets/SnapShotExperiment2015/CEUTrio/Alignments/G94982.NA12878/NA12878.cram"
samfile = pysam.AlignmentFile(cram, "rc", reference_filename = grch38)	# "rb" for bam, "rc" for cram

tr_file_name="/dsde/data/deep/takutoencoder/x_tr.npy"
test_file_name="/dsde/data/deep/takutoencoder/x_test.npy"

In [4]:
#### Set parameters
p_sampling = 0.5
num_tr_examples = int(1e6)
num_test_examples = int(1e4)
read_length = 151

In [42]:
#### Helper Functions

def qc_read(read):
    ''' Given a read decide whether it's a good read '''
    read_group = read.get_tag('RG')
    if 'artificial' in read_group.lower():
        return False
    elif not read.is_proper_pair or not read.is_paired:
        return False
    elif read.is_duplicate or read.is_secondary or read.is_supplementary or read.is_qcfail or read.is_unmapped:
        return False
    
    return True

In [38]:
#### Training
contig = "chr1"
X_tr = np.zeros((num_tr_examples, read_length))
X_test = np.zeros((num_test_examples, read_length))
i=0
j=0

for read in samfile.fetch(contig):
    if i == num_tr_examples + num_test_examples:
        break;
    
    # QC the reads
    if not qc_read(read):
        continue  

    # randomly sample from among the good reads
    dice = np.random.rand()
    if dice > p_sampling:
        continue    
        
    bqs = np.array(list(reversed(read.query_qualities))) if read.is_reverse else np.array(read.query_qualities)
    if (i < num_tr_examples):
        X_tr[i] = bqs
        i += 1
        continue
    else: 
        X_test[j] = bqs
        j += 1
        i += 1

np.save(tr_file_name, X_tr)
np.save(test_file_name, X_test)

ValueError: could not broadcast input array from shape (101) into shape (151)

In [19]:
print X_tr[num_tr_examples-1]
print X_tr.shape

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
(1000000, 151)


In [5]:
# if the test data has been created already, start here
X_tr = np.load(tr_file_name)
X_test = np.load(test_file_name)

In [6]:
# take the sample code from https://blog.keras.io/building-autoencoders-in-keras.html

# this is the size of our encoded representations
encoding_dim = 32

input_qualities = Input(shape=(read_length,))
encoded = Dense(encoding_dim, activation='relu')(input_qualities)
decoded = Dense(read_length, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_qualities, decoded)

In [7]:
# separate encoder model
encoder = Model(input_qualities, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))

# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]

# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [8]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [9]:
autoencoder.fit(X_tr, X_tr,
                epochs=4,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 1000000 samples, validate on 10000 samples
Epoch 1/10
1000000/1000000 [==============================] - 7s 7us/step - loss: 23.0547 - val_loss: 14.5792
Epoch 2/10
1000000/1000000 [==============================] - 6s 6us/step - loss: 16.9358 - val_loss: 15.0257
Epoch 3/10
1000000/1000000 [==============================] - 6s 6us/step - loss: 16.6843 - val_loss: 14.1843
Epoch 4/10
1000000/1000000 [==============================] - 6s 6us/step - loss: 15.1918 - val_loss: 13.4349
Epoch 5/10
1000000/1000000 [==============================] - 7s 7us/step - loss: 15.9729 - val_loss: 14.2730
Epoch 6/10
1000000/1000000 [==============================] - 7s 7us/step - loss: 16.8950 - val_loss: 15.7329
Epoch 7/10
1000000/1000000 [==============================] - 7s 7us/step - loss: 17.4059 - val_loss: 15.0702
Epoch 8/10
1000000/1000000 [==============================] - 7s 7us/step - loss: 17.5909 - val_loss: 16.8396
Epoch 9/10
1000000/1000000 [==============================] - 7s 7us

In [10]:
model_path = "/Users/tsato/workspace/dsde-deep-learning/takutoencoder/vanilla.h5"
autoencoder.save(model_path)

In [14]:
autoencoder.summary()
encoder.summary()
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 151)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4864      
_________________________________________________________________
dense_2 (Dense)              (None, 151)               4983      
Total params: 9,847
Trainable params: 9,847
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 151)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4864      
Total params: 4,864
Trainable params: 4,864
Non-trainable params: 0
______

In [44]:
N_plot = 300
X = np.zeros((N_plot, read_length))
Y = np.zeros((N_plot, read_length))
i = 0
start_pos = int(30e6) # advance enough reads so we don't reuse same data as training 
for read in samfile.fetch('chr2'):
    if (i >= N_plot):
        break
    # must change from (151,) to (1, 151)...there has got to be a better way?
    orig_bqs = np.reshape(np.array(read.query_qualities), (1, read_length))
    new_bqs = autoencoder.predict(orig_bqs)
    X[i] = orig_bqs
    Y[i] = new_bqs
    i += 1

In [ ]:
# compress and write a new sam file
new_samfile_path = "/dsde/data/deep/takutoencoder/output/NA12878.cram"
new_samfile = pysam.AlignmentFile(new_samfile_path, "wc", template=samfile, reference_filename = grch38)

i = 0
for read in samfile.fetch('chr22'):
    if qc_read(read):  
        orig_bqs = np.reshape(np.array(read.query_qualities), (1, read_length))
        new_bqs = autoencoder.predict(orig_bqs).reshape(151).astype(int).clip(min = 2)
        read.query_qualities = array('b', new_bqs)
        i += 1
    new_samfile.write(read)

## print samfile
/dsde/data/deep/takutoencoder/output/NA12878.cra

In [25]:
wuns = np.ones(15, dtype = int)
array('b', wuns)

array('b', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
np.ones(15, dtype=int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])